# Day 1: LangGraph Foundations & Type-Safe Development

**Duration**: 2 hours (30 min theory + 60 min code + 30 min exercises)

Welcome to the LangGraph Multi-Agent Systems Course! Today we'll build a solid foundation in LangGraph development with a focus on type safety using Pydantic.

## Learning Objectives
By the end of this session, you will:
- Understand multi-agent systems and graph architecture
- Set up a complete LangGraph development environment with OpenAI
- Implement type-safe agents using Pydantic models
- Work with structured outputs and function calling
- Build your first LangGraph agent with error handling

## Prerequisites
- Python 3.8+ installed
- OpenAI API key
- Basic understanding of Python and async programming

---
# 📚 Part 1: Learning Materials (30 minutes)

## Video Resources

Before we dive into coding, watch these essential videos:

### Core Concepts
- **[LangGraph Introduction](https://www.youtube.com/watch?v=hvAPnpSfSGo)** - Official introduction to LangGraph architecture
- **[Multi-Agent Systems Explained](https://www.youtube.com/watch?v=lNFGKd7RzVU)** - Understanding agent communication patterns
- **[Pydantic for Type Safety](https://www.youtube.com/watch?v=502XOB0u8OY)** - Building robust Python applications

### Documentation Links
- [LangGraph Official Documentation](https://langchain-ai.github.io/langgraph/)
- [OpenAI API Documentation](https://platform.openai.com/docs/api-reference)
- [Pydantic Documentation](https://docs.pydantic.dev/latest/)

## Theory: Multi-Agent Systems Introduction

### What are Multi-Agent Systems?
Multi-agent systems consist of multiple autonomous agents that interact with each other to solve complex problems. Each agent has:
- **Autonomy**: Can act independently
- **Reactivity**: Responds to environment changes
- **Proactivity**: Takes initiative to achieve goals
- **Social ability**: Communicates with other agents

### Benefits of Multi-Agent Architecture
1. **Modularity**: Each agent handles specific tasks
2. **Scalability**: Add agents as needed
3. **Robustness**: System continues if one agent fails
4. **Specialization**: Agents can be expert in specific domains
5. **Parallel Processing**: Multiple agents work simultaneously

### LangGraph vs Traditional Approaches

| Traditional Chains | LangGraph Approach |
|-------------------|--------------------|
| Linear execution | Graph-based routing |
| Fixed flow | Dynamic decision making |
| Single agent | Multi-agent coordination |
| Limited state | Rich state management |
| No cycles | Supports loops and cycles |

## Theory: Type Safety with Pydantic

### Why Type Safety Matters
- **Runtime Safety**: Catch errors before they happen
- **Better Developer Experience**: IDE support and autocomplete
- **Documentation**: Types serve as living documentation
- **Refactoring**: Confident code changes
- **Integration**: Better API contracts

### Pydantic Benefits for LangGraph
1. **Data Validation**: Automatic input/output validation
2. **Serialization**: Easy JSON conversion
3. **Schema Generation**: Automatic API schemas
4. **Type Hints**: Full IDE support
5. **Error Handling**: Clear validation error messages

---
# 💻 Part 2: Hands-on Code (60 minutes)

## Environment Setup and Dependencies

Let's start by installing and configuring everything we need.

In [5]:
# Install required packages
!pip install -q langgraph langchain-openai pydantic python-dotenv

In [6]:
# Import essential libraries
import os
import json
import asyncio
from typing import List, Dict, Any, Optional, Literal
from datetime import datetime
from dotenv import load_dotenv

# Pydantic imports for type safety
from pydantic import BaseModel, Field, ValidationError

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# LangChain and OpenAI imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_core.pydantic_v1 import BaseModel as LangChainBaseModel

print("✅ All imports successful!")

✅ All imports successful!


## OpenAI API Configuration

Set up your OpenAI API key. You can get one from [OpenAI Platform](https://platform.openai.com/api-keys).

In [ ]:
# Load environment variables
load_dotenv()
import os
from pathlib import Path
from dotenv import load_dotenv

# Point directly to your project .env (avoids CWD issues)
load_dotenv(Path("/Users/harshitgulati/Coding Projects/Langraph/.env"), override=True)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY missing; check .env path and key name")

# Initialize OpenAI client
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.1,
    timeout=30
)

# Test the connection
try:
    response = llm.invoke([HumanMessage(content="Say 'Hello from LangGraph!'")])
    print(f"✅ OpenAI connection successful: {response.content}")
except Exception as e:
    print(f"❌ OpenAI connection failed: {e}")
    print("Please check your API key and internet connection.")

❌ OpenAI connection failed: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************-UAA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
Please check your API key and internet connection.


## Pydantic Models for Agent State Management

Let's create type-safe state models that will form the foundation of our agents.

In [ ]:
# Base state model for all agents
class AgentState(BaseModel):
    """Base state model with common fields for all agents."""
    
    messages: List[Dict[str, str]] = Field(
        default_factory=list,
        description="Conversation history"
    )
    current_task: Optional[str] = Field(
        default=None,
        description="Current task being processed"
    )
    user_id: Optional[str] = Field(
        default=None,
        description="User identifier for session management"
    )
    timestamp: datetime = Field(
        default_factory=datetime.now,
        description="Last update timestamp"
    )
    error: Optional[str] = Field(
        default=None,
        description="Last error message if any"
    )
    
    class Config:
        # Allow arbitrary types for datetime
        arbitrary_types_allowed = True
        # Enable validation on assignment
        validate_assignment = True


# Specialized state for research tasks
class ResearchState(AgentState):
    """Specialized state for research agents."""
    
    research_query: Optional[str] = Field(
        default=None,
        description="Current research query"
    )
    findings: List[str] = Field(
        default_factory=list,
        description="Research findings collected"
    )
    sources: List[str] = Field(
        default_factory=list,
        description="Source URLs or references"
    )
    confidence: float = Field(
        default=0.0,
        ge=0.0,
        le=1.0,
        description="Confidence score (0-1)"
    )


# Test the models
print("Testing Pydantic models...")

# Create a basic state
basic_state = AgentState(
    messages=[{"role": "user", "content": "Hello!"}],
    current_task="greeting",
    user_id="user123"
)
print(f"✅ Basic state: {basic_state.current_task}")

# Create a research state
research_state = ResearchState(
    messages=[{"role": "user", "content": "Research AI trends"}],
    research_query="Latest AI trends 2024",
    confidence=0.8
)
print(f"✅ Research state: {research_state.research_query}")

# Test validation
try:
    invalid_state = ResearchState(confidence=1.5)  # Should fail
except ValidationError as e:
    print(f"✅ Validation working: {e.errors()[0]['msg']}")

## Structured Output with OpenAI Function Calling

Let's implement structured outputs using OpenAI's function calling capabilities.

In [ ]:
# Define structured output schemas
class WeatherInfo(BaseModel):
    """Structured weather information."""
    
    location: str = Field(description="City and country")
    temperature: float = Field(description="Temperature in Celsius")
    condition: str = Field(description="Weather condition")
    humidity: int = Field(ge=0, le=100, description="Humidity percentage")
    recommendation: str = Field(description="Clothing recommendation")


class TaskAnalysis(BaseModel):
    """Structured task analysis."""
    
    task_type: Literal["research", "coding", "writing", "analysis", "other"] = Field(
        description="Type of task identified"
    )
    complexity: Literal["low", "medium", "high"] = Field(
        description="Task complexity level"
    )
    estimated_time: int = Field(
        ge=1,
        description="Estimated time in minutes"
    )
    required_tools: List[str] = Field(
        description="List of tools needed"
    )
    steps: List[str] = Field(
        description="Breakdown of task steps"
    )


# Function to get structured weather (simulated)
@tool
def get_weather_structured(location: str) -> WeatherInfo:
    """Get structured weather information for a location."""
    # Simulated weather data
    weather_data = {
        "new york": {"temp": 15, "condition": "Cloudy", "humidity": 65},
        "london": {"temp": 8, "condition": "Rainy", "humidity": 80},
        "tokyo": {"temp": 22, "condition": "Sunny", "humidity": 45},
        "sydney": {"temp": 25, "condition": "Partly Cloudy", "humidity": 55}
    }
    
    city_key = location.lower()
    if city_key in weather_data:
        data = weather_data[city_key]
        return WeatherInfo(
            location=location.title(),
            temperature=data["temp"],
            condition=data["condition"],
            humidity=data["humidity"],
            recommendation=f"Dress for {data['condition'].lower()} weather at {data['temp']}°C"
        )
    else:
        return WeatherInfo(
            location=location,
            temperature=20,
            condition="Unknown",
            humidity=50,
            recommendation="Check local weather sources"
        )


# Function for task analysis using LLM
async def analyze_task_structured(task_description: str) -> TaskAnalysis:
    """Analyze a task and return structured information."""
    
    prompt = f"""
    Analyze the following task and provide structured information:
    
    Task: {task_description}
    
    Please analyze:
    1. What type of task this is
    2. The complexity level
    3. Estimated time needed
    4. What tools might be required
    5. Step-by-step breakdown
    
    Respond in JSON format matching the TaskAnalysis schema.
    """
    
    try:
        response = await llm.ainvoke([HumanMessage(content=prompt)])
        
        # Parse the LLM response into structured format
        # For demo purposes, we'll create a structured response
        if "research" in task_description.lower():
            return TaskAnalysis(
                task_type="research",
                complexity="medium",
                estimated_time=30,
                required_tools=["web_search", "summarizer"],
                steps=[
                    "Define research scope",
                    "Search for relevant sources",
                    "Analyze findings",
                    "Synthesize conclusions"
                ]
            )
        else:
            return TaskAnalysis(
                task_type="other",
                complexity="low",
                estimated_time=15,
                required_tools=["basic_tools"],
                steps=["Understand requirements", "Execute task", "Verify results"]
            )
            
    except Exception as e:
        print(f"Error in task analysis: {e}")
        return TaskAnalysis(
            task_type="other",
            complexity="unknown",
            estimated_time=0,
            required_tools=[],
            steps=[]
        )


# Test structured outputs
print("Testing structured outputs...")

# Test weather function
weather = get_weather_structured("New York")
print(f"✅ Weather for {weather.location}: {weather.temperature}°C, {weather.condition}")
print(f"   Recommendation: {weather.recommendation}")

# Test task analysis
task_analysis = await analyze_task_structured("Research the latest AI developments")
print(f"✅ Task analysis: {task_analysis.task_type} task, {task_analysis.complexity} complexity")
print(f"   Estimated time: {task_analysis.estimated_time} minutes")
print(f"   Steps: {', '.join(task_analysis.steps)}")

## First Type-Safe Agent with GPT-4

Now let's build our first complete LangGraph agent with type safety.

In [ ]:
# Enhanced agent state for our first agent
class AssistantState(AgentState):
    """State for our AI assistant agent."""
    
    intent: Optional[str] = Field(
        default=None,
        description="Detected user intent"
    )
    response: Optional[str] = Field(
        default=None,
        description="Agent's response"
    )
    tools_used: List[str] = Field(
        default_factory=list,
        description="Tools used in this interaction"
    )
    confidence_score: float = Field(
        default=0.0,
        ge=0.0,
        le=1.0,
        description="Confidence in the response"
    )


# Agent functions with proper error handling
async def intent_classifier(state: AssistantState) -> AssistantState:
    """Classify user intent from the last message."""
    try:
        if not state.messages:
            return state
        
        last_message = state.messages[-1]
        user_input = last_message.get("content", "")
        
        # Simple intent classification
        intent_prompt = f"""
        Classify the intent of this user message into one of these categories:
        - weather: asking about weather
        - task_analysis: asking to analyze a task
        - greeting: saying hello or greeting
        - question: asking a general question
        - other: anything else
        
        User message: {user_input}
        
        Respond with just the category name.
        """
        
        response = await llm.ainvoke([HumanMessage(content=intent_prompt)])
        intent = response.content.strip().lower()
        
        # Update state
        state.intent = intent
        state.timestamp = datetime.now()
        
        print(f"🎯 Intent classified: {intent}")
        return state
        
    except Exception as e:
        state.error = f"Intent classification error: {str(e)}"
        state.intent = "error"
        print(f"❌ Intent classification failed: {e}")
        return state


async def response_generator(state: AssistantState) -> AssistantState:
    """Generate appropriate response based on intent."""
    try:
        user_input = state.messages[-1].get("content", "") if state.messages else ""
        
        if state.intent == "weather":
            # Extract location and get weather
            location_prompt = f"""
            Extract the location from this weather request. If no specific location is mentioned, use 'New York'.
            Request: {user_input}
            Respond with just the city name.
            """
            
            location_response = await llm.ainvoke([HumanMessage(content=location_prompt)])
            location = location_response.content.strip()
            
            weather_info = get_weather_structured(location)
            state.tools_used.append("weather_tool")
            
            response = f"""
            🌤️ Weather in {weather_info.location}:
            Temperature: {weather_info.temperature}°C
            Condition: {weather_info.condition}
            Humidity: {weather_info.humidity}%
            
            💡 Recommendation: {weather_info.recommendation}
            """
            state.confidence_score = 0.9
            
        elif state.intent == "task_analysis":
            task_info = await analyze_task_structured(user_input)
            state.tools_used.append("task_analyzer")
            
            response = f"""
            📋 Task Analysis:
            Type: {task_info.task_type.title()}
            Complexity: {task_info.complexity.title()}
            Estimated Time: {task_info.estimated_time} minutes
            
            🛠️ Required Tools: {', '.join(task_info.required_tools)}
            
            📝 Steps:
            {chr(10).join([f"{i+1}. {step}" for i, step in enumerate(task_info.steps)])}
            """
            state.confidence_score = 0.8
            
        elif state.intent == "greeting":
            response = "Hello! I'm your AI assistant. I can help you with weather information, task analysis, and answer various questions. How can I assist you today?"
            state.confidence_score = 1.0
            
        else:
            # General response using LLM
            general_prompt = f"""
            You are a helpful AI assistant. Respond to this user message in a friendly and helpful way:
            
            User: {user_input}
            
            Keep your response concise but informative.
            """
            
            llm_response = await llm.ainvoke([HumanMessage(content=general_prompt)])
            response = llm_response.content
            state.confidence_score = 0.7
        
        # Update state
        state.response = response.strip()
        state.messages.append({
            "role": "assistant",
            "content": state.response
        })
        state.timestamp = datetime.now()
        
        print(f"✨ Response generated (confidence: {state.confidence_score})")
        return state
        
    except Exception as e:
        state.error = f"Response generation error: {str(e)}"
        state.response = "I apologize, but I encountered an error while processing your request. Please try again."
        state.confidence_score = 0.0
        print(f"❌ Response generation failed: {e}")
        return state


# Build the LangGraph
def create_assistant_graph():
    """Create the assistant graph with proper error handling."""
    
    # Create the graph
    workflow = StateGraph(AssistantState)
    
    # Add nodes
    workflow.add_node("classify_intent", intent_classifier)
    workflow.add_node("generate_response", response_generator)
    
    # Set entry point
    workflow.set_entry_point("classify_intent")
    
    # Add edges
    workflow.add_edge("classify_intent", "generate_response")
    workflow.add_edge("generate_response", END)
    
    # Compile the graph
    return workflow.compile()


# Test the agent
print("Creating AI Assistant Agent...")
assistant_graph = create_assistant_graph()

# Test with different types of input
test_messages = [
    "Hello! Nice to meet you.",
    "What's the weather like in London?",
    "I need to analyze this task: Create a data dashboard for sales metrics"
]

for i, message in enumerate(test_messages, 1):
    print(f"\n{'='*50}")
    print(f"Test {i}: {message}")
    print(f"{'='*50}")
    
    # Create initial state
    initial_state = AssistantState(
        messages=[{"role": "user", "content": message}],
        user_id=f"test_user_{i}"
    )
    
    # Run the agent
    try:
        result = await assistant_graph.ainvoke(initial_state)
        
        print(f"Intent: {result.intent}")
        print(f"Tools Used: {result.tools_used}")
        print(f"Confidence: {result.confidence_score}")
        print(f"\nResponse:\n{result.response}")
        
        if result.error:
            print(f"❌ Error: {result.error}")
        else:
            print("✅ Success!")
            
    except Exception as e:
        print(f"❌ Agent execution failed: {e}")

print("\n🎉 First LangGraph agent complete!")

## Function Calling with Pydantic Schemas

Let's explore advanced function calling patterns with proper type safety.

In [ ]:
# Advanced Pydantic schemas for function calling
class SearchQuery(BaseModel):
    """Schema for search queries."""
    
    query: str = Field(description="The search query")
    max_results: int = Field(default=5, ge=1, le=20, description="Maximum number of results")
    category: Optional[Literal["news", "academic", "general", "images"]] = Field(
        default="general",
        description="Search category"
    )
    language: str = Field(default="en", description="Language code")


class SearchResult(BaseModel):
    """Schema for search results."""
    
    title: str = Field(description="Result title")
    url: str = Field(description="Result URL")
    snippet: str = Field(description="Result snippet")
    relevance_score: float = Field(ge=0.0, le=1.0, description="Relevance score")
    source: str = Field(description="Source domain")


class SearchResponse(BaseModel):
    """Schema for complete search response."""
    
    query: str = Field(description="Original query")
    total_results: int = Field(description="Total number of results found")
    results: List[SearchResult] = Field(description="List of search results")
    search_time: float = Field(description="Search time in seconds")
    suggestions: List[str] = Field(default_factory=list, description="Query suggestions")


class CalculationRequest(BaseModel):
    """Schema for calculation requests."""
    
    expression: str = Field(description="Mathematical expression to evaluate")
    precision: int = Field(default=2, ge=0, le=10, description="Decimal precision")
    unit: Optional[str] = Field(default=None, description="Unit of measurement")


class CalculationResult(BaseModel):
    """Schema for calculation results."""
    
    expression: str = Field(description="Original expression")
    result: float = Field(description="Calculation result")
    formatted_result: str = Field(description="Formatted result with unit")
    steps: List[str] = Field(default_factory=list, description="Calculation steps")
    error: Optional[str] = Field(default=None, description="Error message if any")


# Type-safe function implementations
@tool
def search_web(query_data: SearchQuery) -> SearchResponse:
    """Simulate web search with structured input/output."""
    import time
    import random
    
    start_time = time.time()
    
    # Simulated search results
    mock_results = [
        SearchResult(
            title=f"Result for '{query_data.query}' - Article {i+1}",
            url=f"https://example{i+1}.com/article",
            snippet=f"This is a relevant snippet about {query_data.query} from a reliable source.",
            relevance_score=round(random.uniform(0.6, 1.0), 2),
            source=f"example{i+1}.com"
        )
        for i in range(min(query_data.max_results, 3))
    ]
    
    search_time = time.time() - start_time
    
    return SearchResponse(
        query=query_data.query,
        total_results=len(mock_results) * 10,  # Simulate more total results
        results=mock_results,
        search_time=round(search_time, 3),
        suggestions=[f"{query_data.query} tutorial", f"{query_data.query} examples"]
    )


@tool
def calculate_math(calc_request: CalculationRequest) -> CalculationResult:
    """Perform mathematical calculations with error handling."""
    import re
    import math
    
    try:
        # Simple expression evaluation (safe subset)
        expression = calc_request.expression.strip()
        
        # Allow only safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            raise ValueError("Expression contains invalid characters")
        
        # Evaluate safely
        result = eval(expression)
        
        # Format result
        formatted = f"{result:.{calc_request.precision}f}"
        if calc_request.unit:
            formatted += f" {calc_request.unit}"
        
        return CalculationResult(
            expression=expression,
            result=float(result),
            formatted_result=formatted,
            steps=[f"Evaluating: {expression}", f"Result: {result}"]
        )
        
    except Exception as e:
        return CalculationResult(
            expression=calc_request.expression,
            result=0.0,
            formatted_result="Error",
            error=str(e)
        )


# Enhanced agent state for function calling
class FunctionCallingState(AgentState):
    """State for agents with function calling capabilities."""
    
    available_functions: List[str] = Field(
        default_factory=lambda: ["search_web", "calculate_math", "get_weather_structured"],
        description="Available function tools"
    )
    function_calls: List[Dict[str, Any]] = Field(
        default_factory=list,
        description="History of function calls made"
    )
    results: List[Dict[str, Any]] = Field(
        default_factory=list,
        description="Function call results"
    )


# Test the type-safe function calling
print("Testing type-safe function calling...")

# Test search function
search_query = SearchQuery(
    query="LangGraph tutorials",
    max_results=3,
    category="general"
)

search_result = search_web(search_query)
print(f"✅ Search completed in {search_result.search_time}s")
print(f"   Found {search_result.total_results} total results")
print(f"   Top result: {search_result.results[0].title}")

# Test calculation function
calc_request = CalculationRequest(
    expression="(25 + 75) * 0.15",
    precision=2,
    unit="USD"
)

calc_result = calculate_math(calc_request)
print(f"✅ Calculation: {calc_result.expression} = {calc_result.formatted_result}")
if calc_result.error:
    print(f"   Error: {calc_result.error}")

# Test validation errors
try:
    invalid_search = SearchQuery(query="test", max_results=25)  # Should fail
except ValidationError as e:
    print(f"✅ Validation caught error: {e.errors()[0]['msg']}")

print("\n🎉 Function calling with Pydantic schemas complete!")

---
# 🚀 Part 3: Practical Exercises (30 minutes)

Now it's time to put your knowledge to practice! Complete these exercises to reinforce your learning.

## Exercise 1: Weather Agent with Structured Responses

**Objective**: Build a weather agent that can handle multiple cities and provide detailed forecasts.

**Requirements**:
1. Create a `WeatherForecast` Pydantic model with 3-day forecast
2. Implement a function that takes multiple cities
3. Add weather alerts and recommendations
4. Include proper error handling

**Starter Code**:

In [ ]:
# Exercise 1: Your turn to implement!
from typing import List, Optional
from datetime import datetime, timedelta

class DayForecast(BaseModel):
    """Single day weather forecast."""
    # TODO: Add fields for date, temperature (high/low), condition, chance of rain
    pass

class WeatherAlert(BaseModel):
    """Weather alert information."""
    # TODO: Add fields for alert type, severity, description
    pass

class WeatherForecast(BaseModel):
    """Multi-day weather forecast."""
    # TODO: Add fields for location, current weather, 3-day forecast, alerts
    pass

@tool
def get_multi_city_weather(cities: List[str]) -> List[WeatherForecast]:
    """Get weather forecasts for multiple cities."""
    # TODO: Implement this function
    pass

# Test your implementation
# weather_forecasts = get_multi_city_weather(["New York", "London", "Tokyo"])
# for forecast in weather_forecasts:
#     print(f"Weather in {forecast.location}:")
#     # TODO: Print forecast details

### Exercise 1 Solution:

In [ ]:
# Exercise 1 Solution
from datetime import datetime, timedelta
import random

class DayForecast(BaseModel):
    """Single day weather forecast."""
    date: datetime = Field(description="Forecast date")
    high_temp: float = Field(description="High temperature in Celsius")
    low_temp: float = Field(description="Low temperature in Celsius")
    condition: str = Field(description="Weather condition")
    rain_chance: int = Field(ge=0, le=100, description="Chance of rain percentage")
    wind_speed: float = Field(ge=0, description="Wind speed in km/h")
    
    class Config:
        arbitrary_types_allowed = True


class WeatherAlert(BaseModel):
    """Weather alert information."""
    alert_type: Literal["storm", "heat", "cold", "wind", "flood"] = Field(
        description="Type of weather alert"
    )
    severity: Literal["low", "moderate", "high", "extreme"] = Field(
        description="Alert severity level"
    )
    description: str = Field(description="Alert description")
    start_time: datetime = Field(description="Alert start time")
    end_time: datetime = Field(description="Alert end time")
    
    class Config:
        arbitrary_types_allowed = True


class WeatherForecast(BaseModel):
    """Multi-day weather forecast."""
    location: str = Field(description="City and country")
    current_temp: float = Field(description="Current temperature")
    current_condition: str = Field(description="Current weather condition")
    forecast_days: List[DayForecast] = Field(description="3-day forecast")
    alerts: List[WeatherAlert] = Field(default_factory=list, description="Active weather alerts")
    recommendations: List[str] = Field(description="Weather-based recommendations")
    last_updated: datetime = Field(default_factory=datetime.now, description="Last update time")
    
    class Config:
        arbitrary_types_allowed = True


@tool
def get_multi_city_weather(cities: List[str]) -> List[WeatherForecast]:
    """Get weather forecasts for multiple cities."""
    forecasts = []
    
    for city in cities:
        try:
            # Generate mock forecast data
            base_temp = random.randint(5, 30)
            conditions = ["Sunny", "Cloudy", "Rainy", "Partly Cloudy", "Stormy"]
            
            # Create 3-day forecast
            forecast_days = []
            for i in range(3):
                date = datetime.now() + timedelta(days=i)
                temp_variation = random.randint(-5, 5)
                
                day_forecast = DayForecast(
                    date=date,
                    high_temp=base_temp + temp_variation + 3,
                    low_temp=base_temp + temp_variation - 3,
                    condition=random.choice(conditions),
                    rain_chance=random.randint(0, 80),
                    wind_speed=random.uniform(5, 25)
                )
                forecast_days.append(day_forecast)
            
            # Generate alerts (sometimes)
            alerts = []
            if random.random() < 0.3:  # 30% chance of alert
                alert = WeatherAlert(
                    alert_type="storm",
                    severity="moderate",
                    description="Thunderstorms expected in the evening",
                    start_time=datetime.now() + timedelta(hours=6),
                    end_time=datetime.now() + timedelta(hours=12)
                )
                alerts.append(alert)
            
            # Generate recommendations
            recommendations = [
                f"Current temperature is {base_temp}°C - dress accordingly",
                "Check rain forecast before outdoor activities",
                "UV protection recommended during sunny periods"
            ]
            
            forecast = WeatherForecast(
                location=city,
                current_temp=base_temp,
                current_condition=random.choice(conditions),
                forecast_days=forecast_days,
                alerts=alerts,
                recommendations=recommendations
            )
            
            forecasts.append(forecast)
            
        except Exception as e:
            print(f"Error getting weather for {city}: {e}")
            continue
    
    return forecasts


# Test the implementation
print("Testing multi-city weather forecast...")
weather_forecasts = get_multi_city_weather(["New York", "London", "Tokyo"])

for forecast in weather_forecasts:
    print(f"\n🌤️ Weather in {forecast.location}:")
    print(f"Current: {forecast.current_temp}°C, {forecast.current_condition}")
    
    print(f"\n3-Day Forecast:")
    for day in forecast.forecast_days:
        print(f"  {day.date.strftime('%Y-%m-%d')}: {day.high_temp}°C/{day.low_temp}°C, {day.condition} ({day.rain_chance}% rain)")
    
    if forecast.alerts:
        print(f"\n⚠️ Alerts:")
        for alert in forecast.alerts:
            print(f"  {alert.severity.upper()} {alert.alert_type}: {alert.description}")
    
    print(f"\n💡 Recommendations:")
    for rec in forecast.recommendations:
        print(f"  • {rec}")

print("\n✅ Exercise 1 complete!")

## Exercise 2: Type-Safe Research Agent

**Objective**: Build a research agent that can gather, analyze, and synthesize information with proper type safety.

**Requirements**:
1. Create a `ResearchPaper` Pydantic model
2. Implement search and analysis functions
3. Add citation management
4. Create a synthesis function that combines findings

**Starter Code**:

In [ ]:
# Exercise 2: Your turn to implement!

class Citation(BaseModel):
    """Research citation information."""
    # TODO: Add fields for author, title, journal, year, url
    pass

class ResearchPaper(BaseModel):
    """Research paper information."""
    # TODO: Add fields for title, abstract, key_findings, citations, relevance_score
    pass

class ResearchSynthesis(BaseModel):
    """Synthesized research findings."""
    # TODO: Add fields for topic, key_themes, conclusions, recommendations, sources
    pass

@tool
def search_research_papers(topic: str, max_papers: int = 5) -> List[ResearchPaper]:
    """Search for research papers on a topic."""
    # TODO: Implement this function
    pass

@tool
def synthesize_research(papers: List[ResearchPaper], research_question: str) -> ResearchSynthesis:
    """Synthesize findings from multiple papers."""
    # TODO: Implement this function
    pass

# Test your implementation
# papers = search_research_papers("artificial intelligence ethics")
# synthesis = synthesize_research(papers, "What are the main ethical concerns in AI?")
# print(synthesis)

### Exercise 2 Solution:

In [ ]:
# Exercise 2 Solution
import random
from datetime import datetime

class Citation(BaseModel):
    """Research citation information."""
    authors: List[str] = Field(description="List of authors")
    title: str = Field(description="Paper title")
    journal: str = Field(description="Journal or conference name")
    year: int = Field(ge=1900, le=2024, description="Publication year")
    url: Optional[str] = Field(default=None, description="Paper URL")
    doi: Optional[str] = Field(default=None, description="Digital Object Identifier")


class ResearchPaper(BaseModel):
    """Research paper information."""
    title: str = Field(description="Paper title")
    abstract: str = Field(description="Paper abstract")
    key_findings: List[str] = Field(description="Main findings or contributions")
    citation: Citation = Field(description="Citation information")
    relevance_score: float = Field(ge=0.0, le=1.0, description="Relevance to search query")
    keywords: List[str] = Field(description="Paper keywords")
    methodology: Optional[str] = Field(default=None, description="Research methodology")


class ResearchSynthesis(BaseModel):
    """Synthesized research findings."""
    topic: str = Field(description="Research topic")
    research_question: str = Field(description="Original research question")
    key_themes: List[str] = Field(description="Main themes identified")
    conclusions: List[str] = Field(description="Key conclusions")
    recommendations: List[str] = Field(description="Recommendations for future work")
    sources_analyzed: int = Field(description="Number of sources analyzed")
    confidence_level: float = Field(ge=0.0, le=1.0, description="Confidence in synthesis")
    limitations: List[str] = Field(description="Limitations of the analysis")
    created_at: datetime = Field(default_factory=datetime.now, description="Synthesis creation time")
    
    class Config:
        arbitrary_types_allowed = True


@tool
def search_research_papers(topic: str, max_papers: int = 5) -> List[ResearchPaper]:
    """Search for research papers on a topic."""
    # Mock research papers for demonstration
    mock_papers_data = {
        "artificial intelligence ethics": [
            {
                "title": "Ethical Considerations in AI Development",
                "abstract": "This paper examines the ethical implications of artificial intelligence development and deployment in modern society.",
                "key_findings": [
                    "Bias in AI systems can perpetuate social inequalities",
                    "Transparency in AI decision-making is crucial for accountability",
                    "Privacy concerns arise from extensive data collection"
                ],
                "authors": ["Smith, J.", "Johnson, A."],
                "journal": "Journal of AI Ethics",
                "year": 2023,
                "keywords": ["ethics", "artificial intelligence", "bias", "transparency"]
            },
            {
                "title": "Fairness in Machine Learning Algorithms",
                "abstract": "An analysis of fairness metrics and bias mitigation techniques in machine learning systems.",
                "key_findings": [
                    "Multiple fairness definitions can conflict with each other",
                    "Bias mitigation requires ongoing monitoring and adjustment",
                    "Stakeholder involvement is essential for defining fairness"
                ],
                "authors": ["Brown, M.", "Davis, K.", "Wilson, R."],
                "journal": "Machine Learning Conference 2023",
                "year": 2023,
                "keywords": ["fairness", "machine learning", "bias mitigation", "algorithms"]
            }
        ],
        "climate change": [
            {
                "title": "Global Temperature Trends and Climate Modeling",
                "abstract": "Comprehensive analysis of global temperature data and climate model predictions for the next century.",
                "key_findings": [
                    "Global temperatures have risen by 1.1°C since pre-industrial times",
                    "Climate models show continued warming under current emission scenarios",
                    "Arctic regions are warming faster than global average"
                ],
                "authors": ["Climate, R.", "Hansen, J."],
                "journal": "Nature Climate Change",
                "year": 2023,
                "keywords": ["climate change", "temperature", "modeling", "global warming"]
            }
        ]
    }
    
    papers = []
    topic_key = topic.lower()
    
    # Find matching papers
    for key, paper_list in mock_papers_data.items():
        if any(word in topic_key for word in key.split()):
            for paper_data in paper_list[:max_papers]:
                citation = Citation(
                    authors=paper_data["authors"],
                    title=paper_data["title"],
                    journal=paper_data["journal"],
                    year=paper_data["year"],
                    url=f"https://example.com/papers/{paper_data['title'].replace(' ', '_').lower()}",
                    doi=f"10.1000/example.{random.randint(1000, 9999)}"
                )
                
                paper = ResearchPaper(
                    title=paper_data["title"],
                    abstract=paper_data["abstract"],
                    key_findings=paper_data["key_findings"],
                    citation=citation,
                    relevance_score=random.uniform(0.7, 1.0),
                    keywords=paper_data["keywords"],
                    methodology="Quantitative analysis with statistical modeling"
                )
                papers.append(paper)
    
    # If no specific papers found, create generic ones
    if not papers:
        for i in range(min(max_papers, 2)):
            citation = Citation(
                authors=[f"Author{i+1}, X.", f"Researcher{i+1}, Y."],
                title=f"Research on {topic.title()} - Study {i+1}",
                journal="Generic Research Journal",
                year=2023,
                url=f"https://example.com/generic_{i+1}"
            )
            
            paper = ResearchPaper(
                title=f"Research on {topic.title()} - Study {i+1}",
                abstract=f"This study investigates various aspects of {topic} and provides insights into current trends and future directions.",
                key_findings=[
                    f"Key finding 1 about {topic}",
                    f"Important observation regarding {topic}",
                    f"Future implications for {topic} research"
                ],
                citation=citation,
                relevance_score=random.uniform(0.6, 0.9),
                keywords=[topic.lower(), "research", "analysis"],
                methodology="Mixed methods approach"
            )
            papers.append(paper)
    
    return papers[:max_papers]


@tool
def synthesize_research(papers: List[ResearchPaper], research_question: str) -> ResearchSynthesis:
    """Synthesize findings from multiple papers."""
    if not papers:
        return ResearchSynthesis(
            topic="Unknown",
            research_question=research_question,
            key_themes=[],
            conclusions=["No papers available for synthesis"],
            recommendations=["Conduct additional research"],
            sources_analyzed=0,
            confidence_level=0.0,
            limitations=["No data available"]
        )
    
    # Extract common themes
    all_keywords = []
    all_findings = []
    
    for paper in papers:
        all_keywords.extend(paper.keywords)
        all_findings.extend(paper.key_findings)
    
    # Identify key themes (most common keywords)
    keyword_counts = {}
    for keyword in all_keywords:
        keyword_counts[keyword] = keyword_counts.get(keyword, 0) + 1
    
    key_themes = sorted(keyword_counts.keys(), key=lambda x: keyword_counts[x], reverse=True)[:5]
    
    # Generate conclusions based on findings
    conclusions = [
        f"Analysis of {len(papers)} papers reveals consistent themes in {', '.join(key_themes[:3])}",
        "Multiple studies indicate the importance of ongoing research in this area",
        "Cross-study analysis shows both convergent and divergent findings"
    ]
    
    # Generate recommendations
    recommendations = [
        "Future research should address identified gaps in current literature",
        "Longitudinal studies would provide valuable insights",
        "Interdisciplinary approaches may yield novel perspectives",
        "Replication studies are needed to validate current findings"
    ]
    
    # Calculate confidence level based on number of papers and their relevance
    avg_relevance = sum(paper.relevance_score for paper in papers) / len(papers)
    confidence = min(0.9, avg_relevance * (len(papers) / 10))  # Max 0.9 confidence
    
    # Identify limitations
    limitations = [
        f"Analysis based on {len(papers)} papers may not represent full literature",
        "Publication bias may affect representativeness of findings",
        "Synthesis relies on abstract-level analysis"
    ]
    
    if len(papers) < 5:
        limitations.append("Limited number of sources analyzed")
    
    # Determine topic from papers
    topic = papers[0].title.split(" - ")[0] if papers else "Research"
    
    return ResearchSynthesis(
        topic=topic,
        research_question=research_question,
        key_themes=key_themes,
        conclusions=conclusions,
        recommendations=recommendations,
        sources_analyzed=len(papers),
        confidence_level=confidence,
        limitations=limitations
    )


# Test the implementation
print("Testing research agent...")

# Search for papers
papers = search_research_papers("artificial intelligence ethics", max_papers=3)
print(f"\n📚 Found {len(papers)} papers:")
for i, paper in enumerate(papers, 1):
    print(f"{i}. {paper.title} (Relevance: {paper.relevance_score:.2f})")
    print(f"   Authors: {', '.join(paper.citation.authors)}")
    print(f"   Journal: {paper.citation.journal} ({paper.citation.year})")

# Synthesize research
synthesis = synthesize_research(papers, "What are the main ethical concerns in AI development?")

print(f"\n🔬 Research Synthesis:")
print(f"Topic: {synthesis.topic}")
print(f"Research Question: {synthesis.research_question}")
print(f"Sources Analyzed: {synthesis.sources_analyzed}")
print(f"Confidence Level: {synthesis.confidence_level:.2f}")

print(f"\n🎯 Key Themes:")
for theme in synthesis.key_themes:
    print(f"  • {theme}")

print(f"\n💡 Conclusions:")
for conclusion in synthesis.conclusions:
    print(f"  • {conclusion}")

print(f"\n🚀 Recommendations:")
for rec in synthesis.recommendations:
    print(f"  • {rec}")

print(f"\n⚠️ Limitations:")
for limitation in synthesis.limitations:
    print(f"  • {limitation}")

print("\n✅ Exercise 2 complete!")

## Challenge: Custom Pydantic Tools Extension

**Objective**: Create a comprehensive tool system with advanced validation and error handling.

**Requirements**:
1. Create a base `Tool` class with common functionality
2. Implement custom validators using Pydantic
3. Add tool chaining capabilities
4. Include comprehensive error handling and logging
5. Create at least 3 different tool types

**Challenge Code**:

In [ ]:
# Challenge: Advanced Tool System
from abc import ABC, abstractmethod
from typing import Union, Callable, Any
from pydantic import validator, root_validator
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ToolExecutionResult(BaseModel):
    """Result of tool execution."""
    tool_name: str = Field(description="Name of the executed tool")
    success: bool = Field(description="Whether execution was successful")
    result: Any = Field(description="Tool execution result")
    error_message: Optional[str] = Field(default=None, description="Error message if failed")
    execution_time: float = Field(description="Execution time in seconds")
    timestamp: datetime = Field(default_factory=datetime.now, description="Execution timestamp")
    metadata: Dict[str, Any] = Field(default_factory=dict, description="Additional metadata")
    
    class Config:
        arbitrary_types_allowed = True


class ToolInput(BaseModel):
    """Base class for tool inputs with validation."""
    
    @validator('*', pre=True)
    def validate_not_empty_string(cls, v):
        """Ensure string fields are not empty."""
        if isinstance(v, str) and not v.strip():
            raise ValueError("String fields cannot be empty")
        return v


class BaseTool(ABC, BaseModel):
    """Base class for all tools with common functionality."""
    
    name: str = Field(description="Tool name")
    description: str = Field(description="Tool description")
    version: str = Field(default="1.0.0", description="Tool version")
    enabled: bool = Field(default=True, description="Whether tool is enabled")
    max_retries: int = Field(default=3, ge=0, le=10, description="Maximum retry attempts")
    timeout: float = Field(default=30.0, gt=0, description="Timeout in seconds")
    
    class Config:
        arbitrary_types_allowed = True
    
    @abstractmethod
    async def execute(self, input_data: ToolInput) -> ToolExecutionResult:
        """Execute the tool with given input."""
        pass
    
    async def execute_with_retry(self, input_data: ToolInput) -> ToolExecutionResult:
        """Execute tool with automatic retry logic."""
        if not self.enabled:
            return ToolExecutionResult(
                tool_name=self.name,
                success=False,
                result=None,
                error_message="Tool is disabled",
                execution_time=0.0
            )
        
        last_error = None
        for attempt in range(self.max_retries + 1):
            try:
                logger.info(f"Executing {self.name}, attempt {attempt + 1}")
                return await self.execute(input_data)
            except Exception as e:
                last_error = e
                logger.warning(f"Attempt {attempt + 1} failed for {self.name}: {e}")
                if attempt < self.max_retries:
                    continue
        
        return ToolExecutionResult(
            tool_name=self.name,
            success=False,
            result=None,
            error_message=f"Failed after {self.max_retries + 1} attempts: {last_error}",
            execution_time=0.0
        )


# Specific tool implementations
class TextAnalysisInput(ToolInput):
    """Input for text analysis tool."""
    text: str = Field(min_length=1, max_length=10000, description="Text to analyze")
    analysis_type: Literal["sentiment", "keywords", "summary", "language"] = Field(
        description="Type of analysis to perform"
    )
    
    @validator('text')
    def validate_text_content(cls, v):
        """Validate text content."""
        if len(v.split()) < 2:
            raise ValueError("Text must contain at least 2 words")
        return v


class DataProcessingInput(ToolInput):
    """Input for data processing tool."""
    data: List[Dict[str, Any]] = Field(description="Data to process")
    operation: Literal["filter", "aggregate", "transform", "validate"] = Field(
        description="Processing operation"
    )
    parameters: Dict[str, Any] = Field(default_factory=dict, description="Operation parameters")
    
    @validator('data')
    def validate_data_not_empty(cls, v):
        if not v:
            raise ValueError("Data list cannot be empty")
        return v


class APICallInput(ToolInput):
    """Input for API call tool."""
    url: str = Field(description="API endpoint URL")
    method: Literal["GET", "POST", "PUT", "DELETE"] = Field(default="GET", description="HTTP method")
    headers: Dict[str, str] = Field(default_factory=dict, description="Request headers")
    params: Dict[str, Any] = Field(default_factory=dict, description="Request parameters")
    
    @validator('url')
    def validate_url_format(cls, v):
        if not v.startswith(('http://', 'https://')):
            raise ValueError("URL must start with http:// or https://")
        return v


# Tool implementations
class TextAnalysisTool(BaseTool):
    """Tool for analyzing text content."""
    
    def __init__(self, **data):
        super().__init__(
            name="text_analyzer",
            description="Analyzes text for sentiment, keywords, summaries, and language detection",
            **data
        )
    
    async def execute(self, input_data: TextAnalysisInput) -> ToolExecutionResult:
        """Execute text analysis."""
        import time
        start_time = time.time()
        
        try:
            text = input_data.text
            analysis_type = input_data.analysis_type
            
            if analysis_type == "sentiment":
                # Mock sentiment analysis
                positive_words = ["good", "great", "excellent", "amazing", "wonderful"]
                negative_words = ["bad", "terrible", "awful", "horrible", "disappointing"]
                
                words = text.lower().split()
                positive_count = sum(1 for word in words if word in positive_words)
                negative_count = sum(1 for word in words if word in negative_words)
                
                if positive_count > negative_count:
                    sentiment = "positive"
                    score = 0.7
                elif negative_count > positive_count:
                    sentiment = "negative"
                    score = 0.3
                else:
                    sentiment = "neutral"
                    score = 0.5
                
                result = {
                    "sentiment": sentiment,
                    "score": score,
                    "positive_words": positive_count,
                    "negative_words": negative_count
                }
                
            elif analysis_type == "keywords":
                # Mock keyword extraction
                words = text.lower().split()
                word_freq = {}
                for word in words:
                    if len(word) > 3:  # Only consider words longer than 3 characters
                        word_freq[word] = word_freq.get(word, 0) + 1
                
                keywords = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
                result = {"keywords": keywords, "total_words": len(words)}
                
            elif analysis_type == "summary":
                # Mock text summarization
                sentences = text.split('.')
                summary = sentences[0] if sentences else text[:100]
                result = {
                    "summary": summary.strip(),
                    "original_length": len(text),
                    "summary_length": len(summary),
                    "compression_ratio": len(summary) / len(text) if text else 0
                }
                
            elif analysis_type == "language":
                # Mock language detection
                result = {
                    "detected_language": "en",
                    "confidence": 0.95,
                    "alternative_languages": ["es", "fr"]
                }
            
            execution_time = time.time() - start_time
            
            return ToolExecutionResult(
                tool_name=self.name,
                success=True,
                result=result,
                execution_time=execution_time,
                metadata={"analysis_type": analysis_type, "text_length": len(text)}
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            return ToolExecutionResult(
                tool_name=self.name,
                success=False,
                result=None,
                error_message=str(e),
                execution_time=execution_time
            )


class DataProcessingTool(BaseTool):
    """Tool for data processing operations."""
    
    def __init__(self, **data):
        super().__init__(
            name="data_processor",
            description="Processes data with filter, aggregate, transform, and validate operations",
            **data
        )
    
    async def execute(self, input_data: DataProcessingInput) -> ToolExecutionResult:
        """Execute data processing operation."""
        import time
        start_time = time.time()
        
        try:
            data = input_data.data
            operation = input_data.operation
            params = input_data.parameters
            
            if operation == "filter":
                field = params.get("field")
                value = params.get("value")
                if field and value is not None:
                    result = [item for item in data if item.get(field) == value]
                else:
                    result = data
                    
            elif operation == "aggregate":
                field = params.get("field")
                agg_type = params.get("type", "count")
                
                if agg_type == "count":
                    result = {"count": len(data)}
                elif agg_type == "sum" and field:
                    total = sum(item.get(field, 0) for item in data if isinstance(item.get(field), (int, float)))
                    result = {"sum": total}
                elif agg_type == "average" and field:
                    values = [item.get(field) for item in data if isinstance(item.get(field), (int, float))]
                    avg = sum(values) / len(values) if values else 0
                    result = {"average": avg}
                else:
                    result = {"count": len(data)}
                    
            elif operation == "transform":
                field = params.get("field")
                transform_type = params.get("type", "uppercase")
                
                result = []
                for item in data:
                    new_item = item.copy()
                    if field in new_item and isinstance(new_item[field], str):
                        if transform_type == "uppercase":
                            new_item[field] = new_item[field].upper()
                        elif transform_type == "lowercase":
                            new_item[field] = new_item[field].lower()
                    result.append(new_item)
                    
            elif operation == "validate":
                required_fields = params.get("required_fields", [])
                valid_items = []
                invalid_items = []
                
                for item in data:
                    if all(field in item for field in required_fields):
                        valid_items.append(item)
                    else:
                        invalid_items.append(item)
                
                result = {
                    "valid_items": valid_items,
                    "invalid_items": invalid_items,
                    "validation_summary": {
                        "total": len(data),
                        "valid": len(valid_items),
                        "invalid": len(invalid_items)
                    }
                }
            
            execution_time = time.time() - start_time
            
            return ToolExecutionResult(
                tool_name=self.name,
                success=True,
                result=result,
                execution_time=execution_time,
                metadata={"operation": operation, "data_size": len(data)}
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            return ToolExecutionResult(
                tool_name=self.name,
                success=False,
                result=None,
                error_message=str(e),
                execution_time=execution_time
            )


class APICallTool(BaseTool):
    """Tool for making API calls."""
    
    def __init__(self, **data):
        super().__init__(
            name="api_caller",
            description="Makes HTTP API calls with various methods and parameters",
            **data
        )
    
    async def execute(self, input_data: APICallInput) -> ToolExecutionResult:
        """Execute API call."""
        import time
        start_time = time.time()
        
        try:
            # Mock API call (in real implementation, use aiohttp or requests)
            url = input_data.url
            method = input_data.method
            
            # Simulate API response
            mock_response = {
                "status_code": 200,
                "data": {
                    "message": f"Successful {method} request to {url}",
                    "timestamp": datetime.now().isoformat(),
                    "method": method,
                    "url": url
                },
                "headers": {"Content-Type": "application/json"}
            }
            
            execution_time = time.time() - start_time
            
            return ToolExecutionResult(
                tool_name=self.name,
                success=True,
                result=mock_response,
                execution_time=execution_time,
                metadata={"method": method, "url": url}
            )
            
        except Exception as e:
            execution_time = time.time() - start_time
            return ToolExecutionResult(
                tool_name=self.name,
                success=False,
                result=None,
                error_message=str(e),
                execution_time=execution_time
            )


# Tool chaining system
class ToolChain(BaseModel):
    """Chain multiple tools together."""
    
    tools: List[BaseTool] = Field(description="List of tools in the chain")
    name: str = Field(description="Chain name")
    
    async def execute_chain(self, inputs: List[ToolInput]) -> List[ToolExecutionResult]:
        """Execute all tools in the chain."""
        results = []
        
        for i, (tool, input_data) in enumerate(zip(self.tools, inputs)):
            logger.info(f"Executing tool {i+1}/{len(self.tools)} in chain: {tool.name}")
            result = await tool.execute_with_retry(input_data)
            results.append(result)
            
            # Stop chain execution if a tool fails (optional behavior)
            if not result.success:
                logger.error(f"Tool chain '{self.name}' stopped due to failure in {tool.name}")
                break
        
        return results


# Test the advanced tool system
async def test_advanced_tools():
    """Test the advanced tool system."""
    print("🔧 Testing Advanced Tool System")
    print("=" * 50)
    
    # Create tools
    text_tool = TextAnalysisTool()
    data_tool = DataProcessingTool()
    api_tool = APICallTool()
    
    # Test text analysis
    print("\n1. Testing Text Analysis Tool:")
    text_input = TextAnalysisInput(
        text="This is a wonderful and amazing product. I absolutely love it!",
        analysis_type="sentiment"
    )
    
    result = await text_tool.execute_with_retry(text_input)
    print(f"   Success: {result.success}")
    print(f"   Result: {result.result}")
    print(f"   Execution time: {result.execution_time:.3f}s")
    
    # Test data processing
    print("\n2. Testing Data Processing Tool:")
    data_input = DataProcessingInput(
        data=[
            {"name": "Alice", "age": 30, "score": 85},
            {"name": "Bob", "age": 25, "score": 92},
            {"name": "Charlie", "age": 35, "score": 78}
        ],
        operation="aggregate",
        parameters={"field": "score", "type": "average"}
    )
    
    result = await data_tool.execute_with_retry(data_input)
    print(f"   Success: {result.success}")
    print(f"   Result: {result.result}")
    print(f"   Execution time: {result.execution_time:.3f}s")
    
    # Test API call
    print("\n3. Testing API Call Tool:")
    api_input = APICallInput(
        url="https://api.example.com/data",
        method="GET",
        headers={"Authorization": "Bearer token123"}
    )
    
    result = await api_tool.execute_with_retry(api_input)
    print(f"   Success: {result.success}")
    print(f"   Status Code: {result.result['status_code']}")
    print(f"   Execution time: {result.execution_time:.3f}s")
    
    # Test tool chaining
    print("\n4. Testing Tool Chain:")
    tool_chain = ToolChain(
        name="analysis_chain",
        tools=[text_tool, data_tool]
    )
    
    chain_inputs = [
        TextAnalysisInput(text="Great product with excellent features", analysis_type="keywords"),
        DataProcessingInput(
            data=[{"keyword": "great", "count": 5}, {"keyword": "excellent", "count": 3}],
            operation="aggregate",
            parameters={"type": "count"}
        )
    ]
    
    chain_results = await tool_chain.execute_chain(chain_inputs)
    print(f"   Chain executed {len(chain_results)} tools")
    for i, result in enumerate(chain_results):
        print(f"   Tool {i+1}: {result.tool_name} - Success: {result.success}")
    
    # Test validation errors
    print("\n5. Testing Validation Errors:")
    try:
        invalid_input = TextAnalysisInput(text="", analysis_type="sentiment")
    except ValidationError as e:
        print(f"   ✅ Validation error caught: {e.errors()[0]['msg']}")
    
    print("\n🎉 Advanced tool system testing complete!")


# Run the test
await test_advanced_tools()

---
# 🎉 Day 1 Summary

Congratulations! You've completed Day 1 of the LangGraph Multi-Agent Systems Course. Here's what you've accomplished:

## 🏆 Key Achievements

### 1. **Environment Setup**
- ✅ Configured OpenAI API integration
- ✅ Set up LangGraph development environment
- ✅ Installed and configured all necessary dependencies

### 2. **Type Safety Mastery**
- ✅ Created comprehensive Pydantic models for state management
- ✅ Implemented custom validators and error handling
- ✅ Built type-safe tool systems with advanced validation

### 3. **LangGraph Fundamentals**
- ✅ Built your first LangGraph agent with proper architecture
- ✅ Implemented intent classification and response generation
- ✅ Added comprehensive error handling and retry logic

### 4. **Structured Outputs**
- ✅ Mastered OpenAI function calling with Pydantic schemas
- ✅ Created weather and task analysis tools
- ✅ Implemented proper input/output validation

### 5. **Practical Applications**
- ✅ Built a multi-city weather forecast system
- ✅ Created a research paper analysis and synthesis tool
- ✅ Developed an advanced tool chaining system

## 🚀 Next Steps

You're now ready for **Day 2: State Management & Persistence**! Tomorrow you'll learn:
- Graph architecture patterns
- Checkpointer implementations (InMemory, SQLite, PostgreSQL)
- State persistence and recovery
- Advanced routing and conditional logic

## 📚 Additional Resources

For deeper learning, explore:
- [LangGraph Tutorials](https://langchain-ai.github.io/langgraph/tutorials/)
- [Pydantic Advanced Usage](https://docs.pydantic.dev/latest/usage/)
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [Multi-Agent Systems Research](https://arxiv.org/search/?query=multi-agent+systems&searchtype=all)

## 💡 Pro Tips for Tomorrow

1. **Review State Models**: The Pydantic models you created today will be essential for state persistence
2. **Practice Error Handling**: The patterns you learned will be crucial for robust state management
3. **Understand Graph Flow**: Think about how your agents can maintain state across interactions

Great work today! See you in Day 2! 🎯